In [43]:
import pandas as pd
import random 

#### Загрузка дата фрейма

In [44]:
df_train = pd.read_csv('train_sec.csv')
df_test = pd.read_csv('test_sec.csv')
df_train.drop(columns='Unnamed: 0', axis=0, inplace=True)
df_test.drop(columns='Unnamed: 0', axis=0, inplace=True)


#### Предварительная аналитика

In [45]:
df_train['target'].value_counts()# количевство машеников и людей

1    4288
0    1712
Name: target, dtype: int64

Видим что в датасете гораздо больше мошенников, чем обычных людей. Значит, наш датасет плохо сбалансирован(учтём это для построении модели).

In [46]:
df_train.isna().sum() # посчитали количевство NaN в каждой колонке


time_1      11
time_2      13
time_3      12
time_4      17
time_5      12
time_6       6
time_7      10
time_8      14
time_9      14
time_10      0
site_1     738
site_2     697
site_3     741
site_4     712
site_5     738
site_6     719
site_7     691
site_8     706
site_9     713
site_10    707
target       0
dtype: int64

Видим, что пропущено очнь много данных про сайты и не так много про время пребывания на них.

In [47]:
(len(df_train) - len(df_train.dropna()))/len(df_train) 

0.719

Видим что примерно 70% строк содержат пропущенные значения. Это очень много, т.ч. удалять эти строки будет очень плохой идеей. Попробуем заполнить эти данные.

#### Избавление от NaN в дата фрейме

In [49]:
import warnings
import numpy as np
warnings.filterwarnings('ignore')

import pandas as pd
df_train = pd.read_csv('train_sec.csv').drop(columns='Unnamed: 0')

def replace_nan(df):
    df = df.copy()
    arr = [df[df['target'] == 0], df[df['target'] == 1]]
    
    #заменяем сайты
    for target in [0, 1]: #берем отдельно злоумышленников и нет
        for i in range(1, 11):
            null_sites = arr[target][f'site_{i}'].isnull()
            arr[target][f'site_{i}'][null_sites] = arr[target][f'site_{i}'].dropna().sample(null_sites.sum()).values

    #заменяем время на сайтах
    for target in [0, 1]:
        for i in range(1, 11):
            #составим словарь, где каждому сайту сопосотавим среднее время на нем
            mean_time = arr[target].groupby([f'site_{i}'])[f'time_{i}'].agg('mean').reset_index()
            mean_time = mean_time.set_index(f'site_{i}')
            mean_time =  mean_time.to_dict()[f'time_{i}']
            
            #заменим время для каждой группы людей для каждого сайта на соответсвующее среднее
            #(то есть для каждого отдельно сайта)
            func = lambda row: mean_time[row[f'site_{i}']] if np.isnan(row[f'time_{i}']) else row[f'time_{i}']
            arr[target][f'time_{i}'] = arr[target].apply(func, axis = 1)

            
    res = pd.concat(arr, axis=0).sample(frac=1).reset_index(drop=True)   
    return res.dropna() #на всякий случай

df_train = replace_nan(df_train)
df_test = replace_nan(df_test)

In [51]:
df_train['site_1'].value_counts() # подсчёт количевства тех или иных вариантов

2.0    1432
1.0    1310
8.0     870
4.0     683
3.0     653
7.0     621
5.0     294
6.0      79
9.0      58
Name: site_1, dtype: int64